## 可逆圧縮

圧縮後のデータから、元のデータを完全に再現できる圧縮アルゴリズムです。  
代表的なものには ランレングス圧縮(連長圧縮) や ハフマン符号 があげられます。

## 非可逆圧縮

圧縮後のデータから、元のデータを完全に復元できない圧縮アルゴリズムです。  
画像や映像、音声など、人間の認知特性上伝わりずらい部分を大幅に減らし、重要となる部分の情報を多く残すようなアルゴリズムになります。  
逆に一部でも異なれば情報として価値がなくなるもの(テキストデータなど)にはあまり利用されません。

画像では JPEG 形式、音声だと MP3 形式などが代表的なアルゴリズムとしてあげられます。


# ランレングス圧縮(連長圧縮) RLE: Run Length Encoding

ある連続したデータをデータ 1 つ分と連続した長さで表して圧縮します。例えば "AAAAA" を "A" が 5 回続くデータなので、 "A5" という風に圧縮します。

FAX や単純な画像データについて利用されるアルゴリズムです。

<br>

1 つのデータが長く続くデータや、そもそも出現するデータの種類が少ないデータなどは、圧縮効率が高くなる傾向にあります。  
逆に、データがあまり連続しないデータについては、圧縮効率が悪いどころか圧縮前よりデータサイズが大きくなってしまうこともあります。  
例えば "ABCDE" という 5 文字を「データ \* 連続回数」の形に変換すると "A1B1C1D1E1" となり 2 倍のサイズになってしまいます。


# PickBits

ランレングス圧縮は連続しないデータ部分の圧縮効率が悪いどころか、逆にデータ量が増えてしまいます。  
その欠点を改良したものが、**PickBits** と呼ばれるアルゴリズムです。  
基本的な考え方は変わらないのですが、連続しないデータ部分が続く部分をフラグ管理することで圧縮率を高めます。

<br>

"AAABCDEE" という文字を通常のランレングス圧縮で圧縮すると、"A3B1C1D1E2" となり元のデータより大きなデータになってしまいます。  
これを PickBits では次のように圧縮します。

"A3-3BCDE2"

PickBits では連続しないデータが続く部分についてカウントし、「-(カウント) + 連続しないデータ」として表現します。  
上記の例では "-3BCD" の部分に該当します。これは 3 つのデータ "B", "C", "D" について連続しない(つまり 1 文字のみ)という意味になります。


In [16]:

def run_length_encode(S: str):
    result = []

    prev = S[0]
    count = 0
    for char in S:
        if prev == char:
            count += 1
        else:
            result.append((prev, count))
            prev = char
            count = 1
    result.append((prev, count))

    return result


def run_length_decode(li):
    result = ""
    for char, count in li:
        result += char * count

    return result


plain_text = "AAAAADDSSDDDBCDEE"
encoded = run_length_encode(plain_text)
decoded = run_length_decode(encoded)
print("plain text:", plain_text)
print("decoded:", encoded)
print("encoded:", decoded)


plain text: AAAAADDSSDDDBCDEE
decoded: [('A', 5), ('D', 2), ('S', 2), ('D', 3), ('B', 1), ('C', 1), ('D', 1), ('E', 2)]
encoded: AAAAADDSSDDDBCDEE


In [37]:

def pick_bits_encode(S: str):
    res = []

    prev = S[0]
    count = 0
    for char in S:
        if prev == char:
            count += 1
        else:
            res.append((prev, count))
            prev = char
            count = 1
    res.append((prev, count))


    one_count = 0
    pick_chars = ""
    ans = ""
    for char, count in res:
        if count == 1:
            one_count += 1
            pick_chars += char
        else:
            if one_count > 1:
                ans += "-"+str(one_count)+pick_chars
                pick_chars = ""
            one_count = 0

            ans += char+str(count) 

    return ans





def pick_bits_decode(code: str):
    result = ""
    # for char, count in li:
    #     result += char * count
    
    i = 0
    while i < len(code):
        if code[i] == "-":
            num = int(code[i+1])
            result += code[i+2:i+2+num]
            i+=num+2
        else:
            result += code[i]*int(code[i+1])
            i+= 2

    return result


plain_text = "AAABCDEEFRDDERWFADEEEEE"
encoded = pick_bits_encode(plain_text)
decoded = pick_bits_decode(encoded)
print("plain text:", plain_text)
print("decoded:", encoded)
print("encoded:", decoded)

plain text: AAABCDEEFRDDERWFADEEEEE
decoded: A3-3BCDE2-2FRD2-6ERWFADE5
encoded: AAABCDEEFRDDERWFADEEEEE
